In [1]:
%load_ext autoreload
%autoreload 2

$$\begin{aligned}
&\min_X f(X) = \textrm{tr}(A^\top XB X^\top)  \\
& = \textrm{tr}(X^\top A^\top XB) & x = \textrm{vec}(X)\\
& = \left <\textrm{vec}(X),  \textrm{vec}(A^\top X B )  \right > \\
& = \left <\textrm{vec}(X), B^\top \otimes A^\top \cdot \textrm{vec}(X)  \right > \\ 
& = x^\top (B^\top \otimes A^\top) x\\ 
& = \left<AX, XB\right> \\
\mathbf{s.t.} & \\ 
&X \in \Pi_{n}
\end{aligned}$$

In [2]:
from qap_lp.models import *

In [3]:
instance_name = 'chr12a'

A0, B0 = parse(f'qapdata/{instance_name}.dat')
A = A0 / np.linalg.norm(A0)
B = B0 / np.linalg.norm(B0)
n, m = A.shape
e = np.ones(shape=n)
E = np.ones(shape=(n, n))
ab = np.kron(B.T, A.T)

param = QAPParam(A, B, n, m, e, E, ab)
nablaf = QAPDerivative(param)

In [4]:
# x0
x0 = np.eye(n)

## projected gradient

working on problem:
### exact penalty

$$\begin{aligned}
F_{\mu} & =  f  + \mu\cdot | \textrm{tr}(XX^T) -  n| \\
 &= f  + \mu\cdot n - \mu\cdot \textrm{tr}(XX^T)
\end{aligned}$$

very likely to become a concave function, cannot be directly solved by conic solver.

#### derivatives

$$\begin{aligned}
\nabla F_\mu  = A^TXB + AXB^T - 2\mu X
\end{aligned}$$

In [5]:
nabla = QAPDerivativeL2Penalty(param)

In [6]:
nabla.partial_f(x0, 1)

array([[-1.91271179e+00,  6.82274403e-02,  9.44580824e-02,
         9.28928276e-02,  1.30509437e-01,  1.16359028e-01,
         5.74726245e-02,  1.24298909e-01,  1.00466641e-01,
         8.03076734e-02,  1.19022486e-01,  1.14781150e-01],
       [ 7.99794748e-02, -1.89467349e+00,  1.69098018e-01,
         4.28677867e-02,  1.38121120e-01,  8.17971902e-02,
         1.16863949e-01,  9.97218831e-02,  1.31961084e-01,
         7.92978316e-02,  5.22593159e-02,  1.45694933e-01],
       [ 9.82071203e-03,  3.01437793e-02, -1.96081814e+00,
         2.64578566e-02,  1.49456594e-02,  5.17039029e-02,
         1.78994469e-02,  3.15323119e-02,  3.03457477e-02,
         8.17971902e-03,  2.72152380e-02,  4.02926900e-02],
       [ 0.00000000e+00,  1.04518632e-02,  1.56777948e-02,
        -1.99245143e+00,  1.71294424e-02,  2.09037264e-02,
         2.61296580e-03,  9.87120412e-03,  2.29360331e-02,
         4.93560206e-03,  1.33551585e-02,  2.75813056e-02],
       [ 0.00000000e+00,  1.95404399e-02,  2.9310659